### Unsloth

In [1]:
from unsloth import FastLanguageModel
import torch

model_name = "models/Qwen3-4B"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = True, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/mnt/data2/icd10/venv_sft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/mnt/data2/icd10/venv_sft/lib/python3.11/site-packages/unsloth/models/rl_replacements.py:946: UserWarning: You are importing from 'trl.experimental'. APIs here are unstable and may change or be removed without notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  import trl.experimental.openenv.utils as openenv_utils
[unsloth_zoo.log|WARNING]Unsloth: Failed to import trl openenv: No module named 'trl.experimental.openenv'


==((====))==  Unsloth 2025.12.1: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 2. Max memory: 47.493 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using bfloat16 full finetuning which cuts memory usage by 50%.
To enable float32 training, use `float32_mixed_precision = True` during FastLanguageModel.from_pretrained


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.42it/s]


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth: Full finetuning is enabled, so .get_peft_model has no effect


### Data Prep

In [3]:
from datasets import load_dataset

# 1. 讀取您的 CSV 檔案
# 假設您的 CSV 檔案就在當前目錄下，名為 "my_data.csv"
dataset = load_dataset("csv", data_files="gt_1_result.csv", split="train")

# 2. 修改轉換函數，對應您的 CSV 欄位名稱
def generate_conversation(examples):
    # 修改這裡：把 CSV 裡的 "question" 欄位抓出來
    problems  = examples["raw_question"]
    
    # 修改這裡：把 CSV 裡的 "assistant_reply" 欄位抓出來
    solutions = examples["assistant_reply"]
    
    conversations = []
    # 使用 zip 將問題和回答配對
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    
    return { "conversations": conversations }

# 3. 執行轉換
# 這會產生一個新的 "conversations" 欄位，裡面裝好格式化的對話
dataset = dataset.map(generate_conversation, batched=True)

print(dataset[0]["conversations"])

[{'content': 'There is a patient with the following ICD-10 codes:：D500,E538,H548,I10,K521,M810,R270,T474X5A,Y92099,Z9181. The medical record for the Patient is as follows:Sex:   F\n \nService: MEDICINE\n \nAllergies: \nAspirin\n \nAttending: ___.\n \nChief Complaint:\nweakness, diarrhea\n \nMajor Surgical or Invasive Procedure:\nNone\n\n \nHistory of Present Illness:\nMs. ___ is a ___ year-old woman with PMH significant for \nchronic anemia, osteoporosis, hypertension, ataxia, and recent \nL5 fracture in the setting of recurrent falls who presents from \nhome with fatigue and generalized weakness and diarrhea.  \n The patient\'s recent history is notable for the follow:  \n - On ___, she presented with 4 days of LBP s/p fall from \nstanding at which time imaging revealed acute L5 fracture. She \nwas evaluated by Spine team who recommended early mobilization, \npain control, but no brace required. She was evaluated by ___, \nand discharged to ___.  \n - She was discharged home with ___ 

In [4]:
# 這裡確保使用正確的 Chat Template (例如 Qwen, Llama 等)
from unsloth.chat_templates import get_chat_template

# 確保 tokenizer 知道要用哪種格式 (這行通常在載入模型時會做，但保險起見可以加)
tokenizer = get_chat_template(tokenizer, chat_template = "qwen-3") 

SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
你將會得知病人的ICD-10代碼跟病例並回推為什麼病人的ICD-10是如此：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結病人的ICD-10編碼集合。  
</reasoning>  
<answer>  
僅輸出正確的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""
def formatting_prompts_func(examples):
    conversations = examples["conversations"]
    texts = []
    
    for convo in conversations:
        convo.insert(0, {"role": "system", "content": SYSTEM_PROMPT})
        text = tokenizer.apply_chat_template(
                convo, 
                tokenize = False, 
                add_generation_prompt = False
            )
        texts.append(text)
    return { "text" : texts, }

# 執行轉換：這會產生一個新的 "text" 欄位
dataset = dataset.map(formatting_prompts_func, batched = True)

# 檢查結果：現在模型真正看到的是這個樣子
print(dataset[0]["text"])

<|im_start|>system
你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
你將會得知病人的ICD-10代碼跟病例並回推為什麼病人的ICD-10是如此：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結病人的ICD-10編碼集合。  
</reasoning>  
<answer>  
僅輸出正確的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
<|im_end|>
<|im_start|>user
There is a patient with the following ICD-10 codes:：D500,E538,H548,I10,K521,M810,R270,T474X5A,Y92099,Z9181. The medical record for the Patient is as follows:Sex:   F
 
Service: MEDICINE
 
Allergies: 
Aspirin
 
Attending: ___.
 
Chief Complaint:
weakness, diarrhea
 
Major Surgical or Invasive Procedure:
None

 
History of Present Illness:
Ms. ___ is a ___ year-old woman with PMH significant for 
chronic anemia, osteoporosis, hypertension, ataxia, and recent 
L5 fracture in the setting of recurrent falls who presents from 
home with fatigue and generalized weakness and diarrhea.  
 The patient's recent history is notable for the follow:  
 - On ___, she presented with 4 days of LBP s/p fall fr

### Train the model

In [5]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 2e-4, # Reduce to 2ez-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

In [6]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,044 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 4,022,468,096 of 4,022,468,096 (100.00% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.204000
2,2.389200
3,1.749800
4,1.368500
5,1.273100
6,1.535400
7,1.240900
8,1.489200
9,1.374500
10,1.442300


In [ ]:
save_path = f"{model_name}-sft"

## lora 方式儲存
# model.save_pretrained_merged(save_path, tokenizer, save_method="merged_16bit") 

# 直接儲存模型與 tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('models/Qwen3-4B_sft/tokenizer_config.json',
 'models/Qwen3-4B_sft/special_tokens_map.json',
 'models/Qwen3-4B_sft/chat_template.jinja',
 'models/Qwen3-4B_sft/vocab.json',
 'models/Qwen3-4B_sft/merges.txt',
 'models/Qwen3-4B_sft/added_tokens.json',
 'models/Qwen3-4B_sft/tokenizer.json')